# Application: Grading ideas
Create a gradesheet for ideas designed as an EDSL survey.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/expectedparrot/edsl/blob/main/docs/notebooks/grading_experiment.ipynb)

In [1]:
# ! pip install edsl

In [2]:
from edsl import Scenario, Agent, Survey, Model
from edsl.questions import QuestionYesNo, QuestionList

Defines the notion of a "requirement" that is worth some number of points.

In [3]:
class Requirement:
    common_header = """Consider this idea for a {{ type }} business: {{ idea }}."""

    def __init__(self, requirement, grading_instructions, question_name, points):
        self.requirement = requirement
        self.grading_instructions = grading_instructions
        self.question_name = question_name
        self.points = points

    @property
    def question_text(self):
        return (self.common_header + "\n" + self.requirement + "\n" + self.grading_instructions + "\n" 
        + "Does this idea satisfy the requirement?")

    def question(self):
        return QuestionYesNo(
            question_text = self.question_text, 
            question_name = self.question_name)

r1 = Requirement(requirement = "The idea is clear and well-written.", 
    grading_instructions = ".",
    question_name = "clear_idea",
    points = 10)

r2 = Requirement(requirement = "The idea has obvious user benefits.", 
    grading_instructions = "Describe the benefits of different user types.",
    question_name = "user_benefits",
    points = 10)

r3 = Requirement(requirement = "The idea will attract a large number of new users.", 
    grading_instructions = "Explain why.",
    question_name = "new_users",
    points = 10)

requirements = [r1, r2, r3]
survey = Survey([r.question() for r in requirements])

Creates a gradesheet for the requirements:

In [4]:
class Gradesheet:
    def __init__(self, result, requirements):
        self.result = result
        self.requirements = requirements
        self.points = 0
        self._problems = []

    def grade(self):
        for requirement in self.requirements:
            answer = self.result.sub_dicts['answer'].get(requirement.question_name)
            if answer == "Yes":
                self.points += requirement.points
            # if answer == "No":
            explain = self.result.sub_dicts['answer'].get(requirement.question_name + "_comment")
            self._problems.append((requirement.question_name, str(requirement.points), explain))    

        return self.points

    def problems(self):
        return self._problems

    def sheet(self):
        from rich.console import Console
        from rich.table import Table
        console = Console()
        table = Table(show_header=True, header_style="bold magenta")

        # Add columns to the table
        table.add_column("Requirement")
        table.add_column("Points")
        table.add_column("Issue")

        for problem in self._problems:
            table.add_row(*problem)        
            table.add_row("", "", "")  # Add an empty row as a separator between items

        console.print(table)

Generates some ideas for demonstration:

In [5]:
def get_ideas(num_ideas,type=""):
    q = QuestionList(
        question_name = "ideas",
        question_text = f"Draft a list of { num_ideas } business ideas for a { type } business.",
    )
    ideas = q.run(progress_bar=True).select("ideas").to_list()[0]
    return ideas

ideas = get_ideas(5, "startup")
ideas

Output()

['Sustainable Packaging Solutions',
 'AI-powered Personal Health Assistant',
 'Mobile Electric Vehicle Charging Service',
 'Virtual Reality Travel Experiences',
 'Smart Urban Farming Systems']

Create some personas for agents that will grade the ideas, runs the survey with the questions parameterized with the ideas, and then shows the results in gradesheet tables:

In [6]:
personas = [
    "You are a business analyst.",
    "You are a venture capital investor.",
    "You are an employee at a startup."
]

agents = [Agent(traits = {"persona":p} ) for p in personas]

scenarios = [Scenario({"type":"startup", "idea":i}) for i in ideas]

results = survey.by(scenarios).by(agents).run(progress_bar=True)

for i in ideas:
    for p in personas:
        print("Idea: ", i)
        print("Persona: ", p)
        g = Gradesheet(results.filter("scenario.idea == '" + i + "' and agent.persona == '" + p + "'")[0], requirements)
        print("Total points earned: ", g.grade())
        g.sheet()

Output()

Idea:  Sustainable Packaging Solutions
Persona:  You are a business analyst.
Total points earned:  20


┏━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Requirement   ┃ Points ┃ Issue                                                                                  ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ clear_idea    │ 10     │ The idea 'Sustainable Packaging Solutions' is a concept rather than a fully developed  │
│               │        │ business plan. It lacks details on execution, market analysis, competitive             │
│               │        │ differentiation, financial projections, and other critical elements required to        │
│               │        │ satisfy the requirement for a startup business idea.                                   │
│               │        │                                                                                        │
│ user_benefits │ 10     │ Sustainable Packaging Solutions would benefit various user types. For consumers, it    │
│               │        │ offers the peace of mind that they are purchasing products that are environmentally    │
│               │        │ friendly. For businesses, it aligns with corporate social responsibility goals and can │
│               │        │ improve brand image while potentially reducing packaging costs in the long term.       │
│               │        │ Governments and communities benefit from reduced environmental impact and potential    │
│               │        │ waste management cost savings. This idea does satisfy the requirement as it presents   │
│               │        │ clear benefits to different user types.                                                │
│               │        │                                                                                        │
│ new_users     │ 10     │ The idea of Sustainable Packaging Solutions is likely to attract a large number of new │
│               │        │ users because of the growing global concern for the environment and the increasing     │
│               │        │ demand for eco-friendly products. Consumers and businesses are actively seeking ways   │
│               │        │ to reduce their carbon footprint and environmental impact, which makes sustainable     │
│               │        │ packaging an attractive option. Moreover, regulations and policies are increasingly    │
│               │        │ favoring sustainable practices, which further drives demand for such solutions.        │
│               │        │ Therefore, this startup idea does satisfy the requirement of having the potential to   │
│               │        │ attract a large user base.                                                             │
│               │        │                                                                                        │
└───────────────┴────────┴────────────────────────────────────────────────────────────────────────────────────────┘

Idea:  Sustainable Packaging Solutions
Persona:  You are a venture capital investor.
Total points earned:  20


┏━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Requirement   ┃ Points ┃ Issue                                                                                  ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ clear_idea    │ 10     │ The idea 'Sustainable Packaging Solutions' is a concept rather than a fully developed  │
│               │        │ business plan. It requires more detail to satisfy the requirement for a startup        │
│               │        │ business proposal, such as market analysis, business model, revenue streams,           │
│               │        │ competitive advantage, and execution strategy.                                         │
│               │        │                                                                                        │
│ user_benefits │ 10     │ Sustainable Packaging Solutions as a startup business idea indeed has obvious user     │
│               │        │ benefits. For consumers, it offers the peace of mind that comes with purchasing        │
│               │        │ products that have a reduced environmental impact. For businesses, it provides a way   │
│               │        │ to meet increasing consumer demand for eco-friendly products, potentially reducing     │
│               │        │ packaging costs in the long term, and staying ahead of regulatory trends favoring      │
│               │        │ sustainability. Moreover, it can enhance brand image and reputation among              │
│               │        │ environmentally conscious customers. This idea satisfies the requirement of having     │
│               │        │ obvious user benefits.                                                                 │
│               │        │                                                                                        │
│ new_users     │ 10     │ Sustainable Packaging Solutions are increasingly in demand due to a growing global     │
│               │        │ emphasis on environmental responsibility and the reduction of waste. Consumers and     │
│               │        │ businesses alike are seeking ways to minimize their ecological footprint, leading to a │
│               │        │ higher likelihood of attracting a large number of new users who are environmentally    │
│               │        │ conscious or required to comply with sustainability regulations.                       │
│               │        │                                                                                        │
└───────────────┴────────┴────────────────────────────────────────────────────────────────────────────────────────┘

...

┏━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Requirement   ┃ Points ┃ Issue                                                                                  ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ clear_idea    │ 10     │ The statement 'Smart Urban Farming Systems' is a concept rather than a fully developed │
│               │        │ idea. It lacks details on the business model, target market, competitive advantage,    │
│               │        │ and operational plan that would be necessary to evaluate it as a startup business      │
│               │        │ idea.                                                                                  │
│               │        │                                                                                        │
│ user_benefits │ 10     │ Smart Urban Farming Systems offer a variety of benefits for different user types. For  │
│               │        │ individual consumers, these systems can provide fresh, locally-grown produce with a    │
│               │        │ smaller carbon footprint due to reduced transportation needs. It also allows consumers │
│               │        │ to have more control over their food sources, potentially leading to healthier eating  │
│               │        │ habits. For businesses such as restaurants, having access to fresh ingredients that    │
│               │        │ can be grown on-site or nearby can enhance food quality and allow for menu innovation. │
│               │        │ Furthermore, urban farming systems can contribute to the sustainability efforts of a   │
│               │        │ city, reducing urban heat islands and improving air quality. Educational institutions  │
│               │        │ can use these systems as a tool for hands-on learning about sustainable agriculture,   │
│               │        │ biology, and ecology. Therefore, the idea satisfies the requirement as it presents     │
│               │        │ clear benefits to various user types.                                                  │
│               │        │                                                                                        │
│ new_users     │ 10     │ Smart Urban Farming Systems are likely to attract a large number of new users due to   │
│               │        │ the growing interest in sustainability, local food production, and technology          │
│               │        │ integration in agriculture. Urban farming meets the demand for fresh, local produce in │
│               │        │ city environments where traditional farming is not possible. Smart systems add value   │
│               │        │ by optimizing resources, improving crop yields, and simplifying the management of      │
│               │        │ urban farms, making it an appealing solution for modern consumers and aspiring urban   │
│               │        │ farmers.                                                                               │
│               │        │                                                                                        │
└───────────────┴────────┴────────────────────────────────────────────────────────────────────────────────────────┘

Idea:  Smart Urban Farming Systems
Persona:  You are a venture capital investor.
Total points earned:  20


┏━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Requirement   ┃ Points ┃ Issue                                                                                  ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ clear_idea    │ 10     │ The idea of 'Smart Urban Farming Systems' is mentioned, but there is no detailed       │
│               │        │ description or business plan provided to assess whether it satisfies the requirement   │
│               │        │ for a startup business proposal.                                                       │
│               │        │                                                                                        │
│ user_benefits │ 10     │ Smart Urban Farming Systems offer a range of benefits for different user types. For    │
│               │        │ individual consumers, they provide access to fresh, locally-grown produce and the      │
│               │        │ opportunity for personal involvement in food production. For businesses, such as       │
│               │        │ restaurants and grocery stores, they offer a steady supply of fresh ingredients with   │
│               │        │ reduced transportation costs and carbon footprint. For cities, these systems can       │
│               │        │ contribute to sustainability goals, green space management, and local job creation.    │
│               │        │ The idea does satisfy the requirement as it presents clear user benefits for a diverse │
│               │        │ group of stakeholders.                                                                 │
│               │        │                                                                                        │
│ new_users     │ 10     │ Smart Urban Farming Systems could potentially attract a large number of new users due  │
│               │        │ to increasing interest in sustainability, local food production, and innovative        │
│               │        │ agricultural technology. Urban areas have limited space, and smart systems could       │
│               │        │ optimize the use of vertical and indoor spaces for farming. Additionally, as           │
│               │        │ populations become more aware of the environmental impact of traditional farming, the  │
│               │        │ demand for locally grown, sustainable produce is likely to increase. This startup idea │
│               │        │ aligns with these trends and could satisfy the requirement by leveraging technology to │
│               │        │ meet the growing demand for efficient urban agriculture solutions.                     │
│               │        │                                                                                        │
└───────────────┴────────┴────────────────────────────────────────────────────────────────────────────────────────┘

Idea:  Smart Urban Farming Systems
Persona:  You are an employee at a startup.
Total points earned:  20


┏━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Requirement   ┃ Points ┃ Issue                                                                                  ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ clear_idea    │ 10     │ The idea 'Smart Urban Farming Systems' is mentioned but not described in detail,       │
│               │        │ therefore it does not satisfy the requirement for being clear and well-written.        │
│               │        │                                                                                        │
│ user_benefits │ 10     │ Smart Urban Farming Systems would provide multiple benefits for different user types.  │
│               │        │ For individual consumers, it offers access to fresh, locally grown produce, and the    │
│               │        │ satisfaction of participating in sustainable practices. For businesses, it could mean  │
│               │        │ a reliable source of fresh ingredients, potential cost savings, and the ability to     │
│               │        │ market themselves as environmentally friendly. For cities and communities, it can lead │
│               │        │ to greener urban areas, improved local food security, and potential economic           │
│               │        │ development through new green jobs. Therefore, the idea does satisfy the requirement   │
│               │        │ of having obvious user benefits.                                                       │
│               │        │                                                                                        │
│ new_users     │ 10     │ Smart Urban Farming Systems could attract a large number of new users due to the       │
│               │        │ growing interest in sustainability, local food production, and technology integration  │
│               │        │ in daily life. It addresses the increasing demand for fresh, organic produce and the   │
│               │        │ desire to reduce the carbon footprint associated with traditional agriculture. By      │
│               │        │ leveraging technology for efficient use of space and resources, such systems can       │
│               │        │ appeal to city dwellers, environmentally conscious consumers, and those interested in  │
│               │        │ modern farming techniques.                                                             │
│               │        │                                                                                        │
└───────────────┴────────┴────────────────────────────────────────────────────────────────────────────────────────┘